# HDF5图像转视频 - 简化版

快速将HDF5文件中的RGB图像序列转换为并排对比视频

## 使用方法
1. 修改下面cell中的路径配置
2. 运行所有cells（Shift+Enter）
3. 等待视频生成完成

In [ ]:
import h5py
import cv2
import numpy as np
from pathlib import Path
import sys

# ==================== 配置参数 ====================

# HDF5文件路径
H5_PATH = "/projects/p32958/chengxuan/data/robomind_fail/failure_data/135-place_cup_on_blue_bowl_with_left_hand/夹取前碰撞/1135137-2025_03_05_16_57_11/data/trajectory.hdf5"

# 输出目录（默认在HDF5文件同目录下创建videos文件夹）
OUTPUT_DIR = None  # None表示自动创建，或者指定路径如: "/path/to/output"

# 视频参数
FPS = 30                      # 帧率
# 尝试多种编码器，按优先级排列
CODEC_OPTIONS = ['avc1', 'H264', 'X264', 'mp4v', 'XVID']  
ADD_INFO = True               # 是否添加信息覆盖层（帧号、时间戳等）
CAMERAS = ['camera_chest', 'camera_head']  # 要处理的相机

# ==================== 代码开始 ====================

# 自动设置输出目录
if OUTPUT_DIR is None:
    OUTPUT_DIR = str(Path(H5_PATH).parent.parent / 'videos')

print("="*80)
print("🎬 HDF5图像转视频工具")
print("="*80)
print(f"\n输入文件: {H5_PATH}")
print(f"输出目录: {OUTPUT_DIR}")
print(f"帧率: {FPS} FPS")
print(f"相机: {CAMERAS}\n")

In [ ]:
def decode_image(compressed_data):
    """解码压缩的图像数据"""
    if compressed_data is None or len(compressed_data) == 0:
        return None
    
    try:
        img = cv2.imdecode(compressed_data, cv2.IMREAD_COLOR)
        if img is None:
            img = np.frombuffer(compressed_data, dtype=np.uint8)
            if img.size == 2764800:
                img = img.reshape(720, 1280, 3)
            elif img.size == 921600:
                img = img.reshape(480, 640, 3)
            elif img.size == 1228800:
                img = img.reshape(640, 640, 3)
            else:
                return None
        
        if len(img.shape) == 3 and img.shape[2] == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img
    except:
        return None

def add_info_overlay(img, frame_idx, camera_name, task_name, time_val=None):
    """在图像上添加信息覆盖层"""
    img_copy = cv2.cvtColor(img.copy(), cv2.COLOR_RGB2BGR)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.6
    thickness = 2
    color = (255, 255, 255)
    bg_color = (0, 0, 0)
    
    y_offset = 30
    
    # 添加文本信息
    texts = [
        f"Task: {task_name}",
        f"Camera: {camera_name}",
        f"Frame: {frame_idx}"
    ]
    
    if time_val is not None:
        texts.append(f"Time: {time_val:.3f}s")
    
    for text in texts:
        (w, h), _ = cv2.getTextSize(text, font, font_scale, thickness)
        cv2.rectangle(img_copy, (10, y_offset - h - 5), (15 + w, y_offset + 5), bg_color, -1)
        cv2.putText(img_copy, text, (10, y_offset), font, font_scale, color, thickness)
        y_offset += 35
    
    return cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB)

def test_codec(codec, width, height, fps):
    """测试编码器是否可用"""
    try:
        fourcc = cv2.VideoWriter_fourcc(*codec)
        temp_file = '/tmp/test_codec.mp4'
        writer = cv2.VideoWriter(temp_file, fourcc, fps, (width, height))
        is_opened = writer.isOpened()
        writer.release()
        
        # 清理临时文件
        if Path(temp_file).exists():
            Path(temp_file).unlink()
        
        return is_opened
    except:
        return False

def create_video_from_hdf5(h5_path, output_dir, cameras, fps=30, codec_options=None, add_info=True):
    """从HDF5创建并排视频"""
    
    if codec_options is None:
        codec_options = ['avc1', 'H264', 'X264', 'mp4v', 'XVID']
    
    # 创建输出目录
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # 打开HDF5文件
    print("📂 读取HDF5文件...")
    with h5py.File(h5_path, 'r') as f:
        # 获取基本信息
        is_compressed = f.attrs.get('compress', True)
        task_name = f['language_instruction'][()].decode('utf-8') if 'language_instruction' in f else 'Unknown'
        time_seq = f['sim_time_seq'][:] if 'sim_time_seq' in f else None
        
        print(f"   任务: {task_name}")
        print(f"   压缩: {is_compressed}")
        
        # 检查相机
        available_cameras = list(f['observations/rgb_images'].keys())
        print(f"   可用相机: {available_cameras}")
        
        for cam in cameras:
            if cam not in available_cameras:
                raise ValueError(f"相机 {cam} 不可用！")
        
        # 获取帧数
        num_frames_list = [len(f['observations/rgb_images'][cam]) for cam in cameras]
        num_frames = min(num_frames_list)
        print(f"   总帧数: {num_frames}")
        
        # 获取第一帧确定分辨率
        print("\n🖼️  读取第一帧...")
        first_frames = []
        for cam in cameras:
            frame_data = f['observations/rgb_images'][cam][0]
            if is_compressed:
                frame = decode_image(frame_data)
            else:
                frame = frame_data
                if len(frame.shape) == 3 and frame.shape[2] == 3:
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            if frame is None:
                raise RuntimeError(f"无法解码 {cam} 的第一帧")
            first_frames.append(frame)
        
        # 计算输出分辨率
        heights = [f.shape[0] for f in first_frames]
        widths = [f.shape[1] for f in first_frames]
        max_height = max(heights)
        
        # 调整所有帧到相同高度
        adjusted_widths = []
        for h, w in zip(heights, widths):
            if h != max_height:
                aspect_ratio = w / h
                new_width = int(max_height * aspect_ratio)
                adjusted_widths.append(new_width)
            else:
                adjusted_widths.append(w)
        
        total_width = sum(adjusted_widths)
        
        print(f"   输出分辨率: {total_width}x{max_height}")
        
        # 测试编码器
        print("\n🔧 测试编码器...")
        working_codec = None
        for codec in codec_options:
            print(f"   测试 {codec}...", end=' ')
            if test_codec(codec, total_width, max_height, fps):
                print("✅ 可用")
                working_codec = codec
                break
            else:
                print("❌ 不可用")
        
        if working_codec is None:
            raise RuntimeError("所有编码器都不可用！请检查OpenCV安装")
        
        print(f"\n✅ 使用编码器: {working_codec}")
        
        # 创建视频写入器
        suffix = "_with_info" if add_info else ""
        video_file = output_path / f"combined_side_by_side{suffix}.mp4"
        
        print(f"\n🎥 创建视频写入器...")
        print(f"   输出文件: {video_file}")
        
        fourcc = cv2.VideoWriter_fourcc(*working_codec)
        writer = cv2.VideoWriter(str(video_file), fourcc, fps, (total_width, max_height))
        
        if not writer.isOpened():
            raise RuntimeError(f"无法创建视频写入器！")
        
        # 逐帧处理
        print(f"\n⏳ 处理帧 (共{num_frames}帧)...")
        failed_frames = 0
        
        for frame_idx in range(num_frames):
            # 显示进度
            if frame_idx % 10 == 0 or frame_idx == num_frames - 1:
                progress = (frame_idx + 1) / num_frames * 100
                sys.stdout.write(f"\r   进度: {frame_idx+1}/{num_frames} ({progress:.1f}%)")
                sys.stdout.flush()
            
            # 读取所有相机的帧
            frames = []
            skip_frame = False
            
            for cam in cameras:
                frame_data = f['observations/rgb_images'][cam][frame_idx]
                if is_compressed:
                    frame = decode_image(frame_data)
                else:
                    frame = frame_data
                    if len(frame.shape) == 3 and frame.shape[2] == 3:
                        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                if frame is None:
                    skip_frame = True
                    failed_frames += 1
                    break
                
                frames.append(frame)
            
            if skip_frame:
                continue
            
            # 添加信息覆盖
            if add_info:
                time_val = time_seq[frame_idx] if time_seq is not None and frame_idx < len(time_seq) else None
                frames = [add_info_overlay(f, frame_idx, cam, task_name, time_val) 
                         for f, cam in zip(frames, cameras)]
            
            # 调整高度
            resized_frames = []
            for frame, target_width in zip(frames, adjusted_widths):
                if frame.shape[0] != max_height or frame.shape[1] != target_width:
                    frame = cv2.resize(frame, (target_width, max_height))
                resized_frames.append(frame)
            
            # 水平拼接
            combined = np.hstack(resized_frames)
            
            # 转换并写入
            combined_bgr = cv2.cvtColor(combined, cv2.COLOR_RGB2BGR)
            writer.write(combined_bgr)
        
        print()  # 换行
        writer.release()
        
        if failed_frames > 0:
            print(f"\n⚠️  警告: {failed_frames} 帧解码失败已跳过")
        
        print(f"\n✅ 视频生成完成！")
        print(f"   保存位置: {video_file}")
        
        # 获取文件大小
        file_size = video_file.stat().st_size / 1024 / 1024
        print(f"   文件大小: {file_size:.2f} MB")
        
        # 验证视频
        print(f"\n🔍 验证视频文件...")
        cap = cv2.VideoCapture(str(video_file))
        if cap.isOpened():
            actual_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            actual_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            actual_fps = cap.get(cv2.CAP_PROP_FPS)
            actual_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            cap.release()
            
            print(f"   ✅ 视频可以正常打开")
            print(f"   分辨率: {actual_width}x{actual_height}")
            print(f"   帧率: {actual_fps:.2f} FPS")
            print(f"   帧数: {actual_frame_count}")
        else:
            print(f"   ⚠️  警告: 视频生成了但可能无法正常播放")
        
        return str(video_file)

print("✅ 函数定义完成\n")

In [ ]:
# 开始生成视频
video_path = create_video_from_hdf5(
    h5_path=H5_PATH,
    output_dir=OUTPUT_DIR,
    cameras=CAMERAS,
    fps=FPS,
    codec_options=CODEC_OPTIONS,
    add_info=ADD_INFO
)

print("\n" + "="*80)
print("🎉 全部完成！")
print("="*80)
print(f"\n视频文件: {video_path}")
print("\n您可以使用视频播放器打开查看")